# 1. Import library

In [1]:

import keras
from numpy import dstack, hstack
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import LSTM, GRU, SimpleRNN, Bidirectional
from tensorflow.keras.optimizers import SGD, Adam, RMSprop
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, LearningRateScheduler
import tensorflow as tf
from tensorflow.keras.layers import TimeDistributed 

from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import MaxPooling1D
from tensorflow.keras.layers import concatenate

import pandas as pd
import csv
import tensorflow.keras as keras
from numpy import dstack, hstack, mean, std
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Flatten, Dropout, LSTM, GRU, SimpleRNN, Bidirectional, Input, BatchNormalization, ConvLSTM2D, Conv1D, Conv2D, Conv3D,  MaxPooling1D, AveragePooling1D,MaxPooling2D, MaxPooling3D, concatenate, GlobalAveragePooling1D, GlobalMaxPooling1D, TimeDistributed
from  tensorflow.keras.optimizers import SGD, Adam, RMSprop
from  tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, LearningRateScheduler
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix
import time
import itertools
import tensorflow.keras.layers as  Layer
from sklearn.model_selection import StratifiedKFold
from tensorflow.keras import backend as K


In [2]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix
import time
import pandas as pd
import csv
import spidev
from numpy import interp
from time import sleep
import RPi.GPIO as GPIO
import signal
import datetime, time
# from threading import _Timer
import csv
import termios,sys,tty, fcntl, os,select
import threading
from sklearn import preprocessing
import cv2


import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from numpy import mean
from numpy import std
import numpy as np

# 1.1 MIT Multi-Head Self-Attention

In [3]:
import tensorflow.keras as keras
import tensorflow.keras.backend as K


class ScaledDotProductAttention(keras.layers.Layer):
    r"""The attention layer that takes three inputs representing queries, keys and values.
    \text{Attention}(Q, K, V) = \text{softmax}(\frac{Q K^T}{\sqrt{d_k}}) V
    See: https://arxiv.org/pdf/1706.03762.pdf
    """

    def __init__(self,
                 return_attention=False,
                 history_only=False,
                 **kwargs):
        """Initialize the layer.
        :param return_attention: Whether to return attention weights.
        :param history_only: Whether to only use history data.
        :param kwargs: Arguments for parent class.
        """
        super(ScaledDotProductAttention, self).__init__(**kwargs)
        self.supports_masking = True
        self.return_attention = return_attention
        self.history_only = history_only
        self.intensity = self.attention = None

    def get_config(self):
        config = {
            'return_attention': self.return_attention,
            'history_only': self.history_only,
        }
        base_config = super(ScaledDotProductAttention, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

    def compute_output_shape(self, input_shape):
        if isinstance(input_shape, list):
            query_shape, key_shape, value_shape = input_shape
        else:
            query_shape = key_shape = value_shape = input_shape
        output_shape = query_shape[:-1] + value_shape[-1:]
        if self.return_attention:
            attention_shape = query_shape[:2] + (key_shape[1],)
            return [output_shape, attention_shape]
        return output_shape

    def compute_mask(self, inputs, mask=None):
        if isinstance(mask, list):
            mask = mask[0]
        if self.return_attention:
            return [mask, None]
        return mask

    def call(self, inputs, mask=None, **kwargs):
        if isinstance(inputs, list):
            query, key, value = inputs
        else:
            query = key = value = inputs
        if isinstance(mask, list):
            mask = mask[1]
        feature_dim = K.shape(query)[-1]
        e = K.batch_dot(query, key, axes=2) / K.sqrt(K.cast(feature_dim, dtype=K.floatx()))
        if self.history_only:
            query_len, key_len = K.shape(query)[1], K.shape(key)[1]
            indices = K.expand_dims(K.arange(0, key_len), axis=0)
            upper = K.expand_dims(K.arange(0, query_len), axis=-1)
            e -= 10000.0 * K.expand_dims(K.cast(indices > upper, K.floatx()), axis=0)
        if mask is not None:
            e -= 10000.0 * (1.0 - K.cast(K.expand_dims(mask, axis=-2), K.floatx()))
        self.intensity = e
        e = K.exp(e - K.max(e, axis=-1, keepdims=True))
        self.attention = e / K.sum(e, axis=-1, keepdims=True)
        v = K.batch_dot(self.attention, value)
        if self.return_attention:
            return [v, self.attention]
        return v


In [4]:

import tensorflow.keras.backend as K
import tensorflow.keras as keras 
# from keras_self_attention import ScaledDotProductAttention


class MultiHeadAttention(keras.layers.Layer):
    """Multi-head attention layer.
    See: https://arxiv.org/pdf/1706.03762.pdf
    """

    def __init__(self,
                 head_num,
                 activation='relu',
                 use_bias=True,
                 kernel_initializer='glorot_normal',
                 bias_initializer='zeros',
                 kernel_regularizer=None,
                 bias_regularizer=None,
                 kernel_constraint=None,
                 bias_constraint=None,
                 history_only=False,
                 **kwargs):
        """Initialize the layer.
        :param head_num: Number of heads.
        :param activation: Activations for linear mappings.
        :param use_bias: Whether to use bias term.
        :param kernel_initializer: Initializer for linear mappings.
        :param bias_initializer: Initializer for linear mappings.
        :param kernel_regularizer: Regularizer for linear mappings.
        :param bias_regularizer: Regularizer for linear mappings.
        :param kernel_constraint: Constraints for linear mappings.
        :param bias_constraint: Constraints for linear mappings.
        :param history_only: Whether to only use history in attention layer.
        """
        self.supports_masking = True
        self.head_num = head_num
        self.activation = keras.activations.get(activation)
        self.use_bias = use_bias
        self.kernel_initializer = keras.initializers.get(kernel_initializer)
        self.bias_initializer = keras.initializers.get(bias_initializer)
        self.kernel_regularizer = keras.regularizers.get(kernel_regularizer)
        self.bias_regularizer = keras.regularizers.get(bias_regularizer)
        self.kernel_constraint = keras.constraints.get(kernel_constraint)
        self.bias_constraint = keras.constraints.get(bias_constraint)
        self.history_only = history_only

        self.Wq = self.Wk = self.Wv = self.Wo = None
        self.bq = self.bk = self.bv = self.bo = None

        self.intensity = self.attention = None
        super(MultiHeadAttention, self).__init__(**kwargs)

    def get_config(self):
        config = {
            'head_num': self.head_num,
            'activation': keras.activations.serialize(self.activation),
            'use_bias': self.use_bias,
            'kernel_initializer': keras.initializers.serialize(self.kernel_initializer),
            'bias_initializer': keras.initializers.serialize(self.bias_initializer),
            'kernel_regularizer': keras.regularizers.serialize(self.kernel_regularizer),
            'bias_regularizer': keras.regularizers.serialize(self.bias_regularizer),
            'kernel_constraint': keras.constraints.serialize(self.kernel_constraint),
            'bias_constraint': keras.constraints.serialize(self.bias_constraint),
            'history_only': self.history_only,
        }
        base_config = super(MultiHeadAttention, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

    def compute_output_shape(self, input_shape):
        if isinstance(input_shape, list):
            q, k, v = input_shape
            return q[:-1] + (v[-1],)
        return input_shape

    def compute_mask(self, inputs, input_mask=None):
        if isinstance(input_mask, list):
            return input_mask[0]
        return input_mask

    def build(self, input_shape):
        if isinstance(input_shape, list):
            q, k, v = input_shape
        else:
            q = k = v = input_shape
        feature_dim = int(v[-1])
        if feature_dim % self.head_num != 0:
            raise IndexError('Invalid head number %d with the given input dim %d' % (self.head_num, feature_dim))
        self.Wq = self.add_weight(
            shape=(int(q[-1]), feature_dim),
            initializer=self.kernel_initializer,
            regularizer=self.kernel_regularizer,
            constraint=self.kernel_constraint,
            name='%s_Wq' % self.name,
        )
        if self.use_bias:
            self.bq = self.add_weight(
                shape=(feature_dim,),
                initializer=self.bias_initializer,
                regularizer=self.bias_regularizer,
                constraint=self.bias_constraint,
                name='%s_bq' % self.name,
            )
        self.Wk = self.add_weight(
            shape=(int(k[-1]), feature_dim),
            initializer=self.kernel_initializer,
            regularizer=self.kernel_regularizer,
            constraint=self.kernel_constraint,
            name='%s_Wk' % self.name,
        )
        if self.use_bias:
            self.bk = self.add_weight(
                shape=(feature_dim,),
                initializer=self.bias_initializer,
                regularizer=self.bias_regularizer,
                constraint=self.bias_constraint,
                name='%s_bk' % self.name,
            )
        self.Wv = self.add_weight(
            shape=(int(v[-1]), feature_dim),
            initializer=self.kernel_initializer,
            regularizer=self.kernel_regularizer,
            constraint=self.kernel_constraint,
            name='%s_Wv' % self.name,
        )
        if self.use_bias:
            self.bv = self.add_weight(
                shape=(feature_dim,),
                initializer=self.bias_initializer,
                regularizer=self.bias_regularizer,
                constraint=self.bias_constraint,
                name='%s_bv' % self.name,
            )
        self.Wo = self.add_weight(
            shape=(feature_dim, feature_dim),
            initializer=self.kernel_initializer,
            regularizer=self.kernel_regularizer,
            constraint=self.kernel_constraint,
            name='%s_Wo' % self.name,
        )
        if self.use_bias:
            self.bo = self.add_weight(
                shape=(feature_dim,),
                initializer=self.bias_initializer,
                regularizer=self.bias_regularizer,
                constraint=self.bias_constraint,
                name='%s_bo' % self.name,
            )
        super(MultiHeadAttention, self).build(input_shape)

    @staticmethod
    def _reshape_to_batches(x, head_num):
        input_shape = K.shape(x)
        batch_size, seq_len, feature_dim = input_shape[0], input_shape[1], input_shape[2]
        head_dim = feature_dim // head_num
        x = K.reshape(x, (batch_size, seq_len, head_num, head_dim))
        x = K.permute_dimensions(x, [0, 2, 1, 3])
        return K.reshape(x, (batch_size * head_num, seq_len, head_dim))

    @staticmethod
    def _reshape_attention_from_batches(x, head_num):
        input_shape = K.shape(x)
        batch_size, seq_len, feature_dim = input_shape[0], input_shape[1], input_shape[2]
        x = K.reshape(x, (batch_size // head_num, head_num, seq_len, feature_dim))
        return K.permute_dimensions(x, [0, 2, 1, 3])

    @staticmethod
    def _reshape_from_batches(x, head_num):
        input_shape = K.shape(x)
        batch_size, seq_len, feature_dim = input_shape[0], input_shape[1], input_shape[2]
        x = K.reshape(x, (batch_size // head_num, head_num, seq_len, feature_dim))
        x = K.permute_dimensions(x, [0, 2, 1, 3])
        return K.reshape(x, (batch_size // head_num, seq_len, feature_dim * head_num))

    @staticmethod
    def _reshape_mask(mask, head_num):
        if mask is None:
            return mask
        seq_len = K.shape(mask)[1]
        mask = K.expand_dims(mask, axis=1)
        mask = K.tile(mask, [1, head_num, 1])
        return K.reshape(mask, (-1, seq_len))

    def call(self, inputs, mask=None):
        if isinstance(inputs, list):
            q, k, v = inputs
        else:
            q = k = v = inputs
        if isinstance(mask, list):
            q_mask, k_mask, v_mask = mask
        else:
            q_mask = k_mask = v_mask = mask
        q = K.dot(q, self.Wq)
        k = K.dot(k, self.Wk)
        v = K.dot(v, self.Wv)
        if self.use_bias:
            q += self.bq
            k += self.bk
            v += self.bv
        if self.activation is not None:
            q = self.activation(q)
            k = self.activation(k)
            v = self.activation(v)
        scaled_dot_product_attention = ScaledDotProductAttention(
            history_only=self.history_only,
            name='%s-Attention' % self.name,
        )
        y = scaled_dot_product_attention(
            inputs=[
                self._reshape_to_batches(q, self.head_num),
                self._reshape_to_batches(k, self.head_num),
                self._reshape_to_batches(v, self.head_num),
            ],
            mask=[
                self._reshape_mask(q_mask, self.head_num),
                self._reshape_mask(k_mask, self.head_num),
                self._reshape_mask(v_mask, self.head_num),
            ],
        )
        self.intensity = self._reshape_attention_from_batches(scaled_dot_product_attention.intensity, self.head_num)
        self.attention = self._reshape_attention_from_batches(scaled_dot_product_attention.attention, self.head_num)
        y = self._reshape_from_batches(y, self.head_num)
        y = K.dot(y, self.Wo)
        if self.use_bias:
            y += self.bo
        if self.activation is not None:
            y = self.activation(y)
        # if TF_KERAS:
        #     # Add shape information to tensor when using `tf.keras`
        input_shape = [K.int_shape(q), K.int_shape(k), K.int_shape(v)]
        output_shape = self.compute_output_shape(input_shape)
        if output_shape[1] is not None:
            output_shape = (-1,) + output_shape[1:]
            y = K.reshape(y, output_shape)
        return y


# 2. Load Dataset

In [ ]:

data_filenames = ['Training_data/0/data_0_22_v7.csv','Training_data/0/data_0_22_2_v7.csv','Training_data/0/data_0_121_s1_t1_1_v7.csv', 'Training_data/0/data_0_211_s1_t1_1_v7.csv', 
                  'Training_data/0/data_0_220_s1_t1_1_v7.csv', 'Training_data/0/data_0_221_s1_t1_1_v7.csv', 
                  'Training_data/1/data_1_011_021_v7.csv','Training_data/1/data_1_120_122_v7.csv', 'Training_data/1/data_1_021_s1_t1_1_v7.csv',
                  'Training_data/1/data_1_120_s1_t1_1_v7.csv','Training_data/1/data_1_122_s1_t1_1_v7.csv', 'Training_data/1/data_1_021_s2_t1_1_v7.csv',
                  'Training_data/2/data_2_22_v7.csv','Training_data/2/data_2_22_2_v7.csv','Training_data/2/data_2_121_s1_t1_1_v7.csv','Training_data/2/data_2_211_s1_t1_1_v7.csv',
                  'Training_data/2/data_2_220_s1_t1_1_v7.csv','Training_data/2/data_2_221_s1_t1_1_v7.csv','Training_data/2/data_2_222_s1_t1_1_v7.csv',
                  'Training_data/3/data_3_22_v7.csv','Training_data/3/data_3_22_2_v7.csv','Training_data/3/data_3_121_s1_t1_1_v7.csv','Training_data/3/data_3_121_s2_t1_1_v7.csv',
                  'Training_data/3/data_3_220_s1_t1_1_v7.csv','Training_data/3/data_3_220_s2_t1_1_v7.csv','Training_data/3/data_3_221_s1_t1_1_v7.csv','Training_data/3/data_3_221_s2_t1_1_v7.csv',  
                  'Training_data/4/data_4_221_v7.csv', 'Training_data/4/data_4_221_2_v7.csv','Training_data/4/data_4_121_s1_t1_1_v7.csv','Training_data/4/data_4_121_s2_t1_1_v7.csv',
                  'Training_data/4/data_4_211_s1_t1_1_v7.csv', 'Training_data/4/data_4_211_s2_t1_1_v7.csv', 'Training_data/4/data_4_210_s1_t1_1_v7.csv','Training_data/4/data_4_210_s2_t1_1_v7.csv',
                  'Training_data/4/data_4_220_s1_t1_1_v7.csv','Training_data/4/data_4_220_s2_t1_1_v7.csv','Training_data/4/data_4_221_s1_t1_1_v7.csv','Training_data/4/data_4_221_s2_t1_1_v7.csv',
                  'Training_data/5/data_5_221_v7.csv', 'Training_data/5/data_5_221_2_v7.csv', 'Training_data/5/data_5_121_s1_t1_1_v7.csv','Training_data/5/data_5_121_s1_t2_1_v7.csv',
                  'Training_data/5/data_5_220_s1_t1_1_v7.csv','Training_data/5/data_5_220_s1_t2_1_v7.csv','Training_data/5/data_5_221_s1_t1_1_v7.csv','Training_data/5/data_5_221_s1_t2_1_v7.csv',
                  'Training_data/6/data_6_121_v7.csv', 'Training_data/6/data_6_221_220_v7.csv', 'Training_data/6/data_6_121_s1_t1_1_v7.csv','Training_data/6/data_6_221_s1_t1_1_v7.csv',
                  'Training_data/7/data_7_221_v7.csv','Training_data/7/data_7_221_3_v7.csv', 'Training_data/7/data_7_221_2_v7.csv', 'Training_data/7/data_7_121_s1_t1_1_v7.csv',
                  'Training_data/7/data_7_121_s2_t1_1_v7.csv','Training_data/7/data_7_121_s3_t1_1_v7.csv', 'Training_data/7/data_7_221_s1_t1_1_v7.csv',
                  'Training_data/7/data_7_221_s2_t1_1_v7.csv','Training_data/7/data_7_221_s3_t1_1_v7.csv',   
                  'Training_data/8/data_8_221_v7.csv', 'Training_data/8/data_8_221_3_v7.csv', 'Training_data/8/data_8_121_s1_t1_1_v7.csv','Training_data/8/data_8_121_s1_t2_1_v7.csv',
                  'Training_data/8/data_8_121_s1_t3_1_v7.csv', 'Training_data/8/data_8_220_s1_t1_1_v7.csv','Training_data/8/data_8_220_s1_t2_1_v7.csv','Training_data/8/data_8_220_s1_t3_1_v7.csv',
                  'Training_data/8/data_8_221_s1_t1_1_v7.csv','Training_data/8/data_8_221_s1_t2_1_v7.csv','Training_data/8/data_8_221_s1_t3_1_v7.csv','Training_data/8/data_8_221_s1_t4_1_v7.csv',
                  'Training_data/8/data_8_221_s1_t5_1_v7.csv','Training_data/8/data_8_121_s1_t4_1_v7.csv',
                  'Training_data/9/data_9_120_v7.csv', 'Training_data/9/data_9_221_220_v7.csv','Training_data/9/data_9_221_220_3_v7.csv',
                  'Training_data/9/data_9_120_s1_t1_1_v7.csv','Training_data/9/data_9_121_s1_t1_1_v7.csv','Training_data/9/data_9_220_s1_t1_1_v7.csv','Training_data/9/data_9_221_s1_t1_1_v7.csv']


label_filenames = ['Training_data/0/data_0_22_v7.csv_label_v1.txt','Training_data/0/data_0_22_2_v7.csv_label_v1.txt','Training_data/0/data_0_121_s1_t1_1_v7.csv_label_v1.txt', 
                    'Training_data/0/data_0_211_s1_t1_1_v7.csv_label_v1.txt', 'Training_data/0/data_0_220_s1_t1_1_v7.csv_label_v1.txt', 'Training_data/0/data_0_221_s1_t1_1_v7.csv_label_v1.txt', 
                  'Training_data/1/data_1_011_021_v7.csv_label_v1.txt','Training_data/1/data_1_120_122_v7.csv_label_v1.txt', 'Training_data/1/data_1_021_s1_t1_1_v7.csv_label_v1.txt',
                  'Training_data/1/data_1_120_s1_t1_1_v7.csv_label_v1.txt','Training_data/1/data_1_122_s1_t1_1_v7.csv_label_v1.txt','Training_data/1/data_1_021_s2_t1_1_v7.csv_label_v1.txt',
                  'Training_data/2/data_2_22_v7.csv_label_v1.txt','Training_data/2/data_2_22_2_v7.csv_label_v1.txt','Training_data/2/data_2_121_s1_t1_1_v7.csv_label_v1.txt','Training_data/2/data_2_211_s1_t1_1_v7.csv_label_v1.txt',
                  'Training_data/2/data_2_220_s1_t1_1_v7.csv_label_v1.txt','Training_data/2/data_2_221_s1_t1_1_v7.csv_label_v1.txt','Training_data/2/data_2_222_s1_t1_1_v7.csv_label_v1.txt',
                  'Training_data/3/data_3_22_v7.csv_label_v1.txt','Training_data/3/data_3_22_2_v7.csv_label_v1.txt','Training_data/3/data_3_121_s1_t1_1_v7.csv_label_v1.txt','Training_data/3/data_3_121_s2_t1_1_v7.csv_label_v1.txt',
                  'Training_data/3/data_3_220_s1_t1_1_v7.csv_label_v1.txt','Training_data/3/data_3_220_s2_t1_1_v7.csv_label_v1.txt','Training_data/3/data_3_221_s1_t1_1_v7.csv_label_v1.txt','Training_data/3/data_3_221_s2_t1_1_v7.csv_label_v1.txt',  
                  'Training_data/4/data_4_221_v7.csv_label_v1.txt', 'Training_data/4/data_4_221_2_v7.csv_label_v1.txt','Training_data/4/data_4_121_s1_t1_1_v7.csv_label_v1.txt','Training_data/4/data_4_121_s2_t1_1_v7.csv_label_v1.txt',
                  'Training_data/4/data_4_211_s1_t1_1_v7.csv_label_v1.txt', 'Training_data/4/data_4_211_s2_t1_1_v7.csv_label_v1.txt', 'Training_data/4/data_4_210_s1_t1_1_v7.csv_label_v1.txt','Training_data/4/data_4_210_s2_t1_1_v7.csv_label_v1.txt',
                  'Training_data/4/data_4_220_s1_t1_1_v7.csv_label_v1.txt','Training_data/4/data_4_220_s2_t1_1_v7.csv_label_v1.txt','Training_data/4/data_4_221_s1_t1_1_v7.csv_label_v1.txt','Training_data/4/data_4_221_s2_t1_1_v7.csv_label_v1.txt',
                  'Training_data/5/data_5_221_v7.csv_label_v1.txt', 'Training_data/5/data_5_221_2_v7.csv_label_v1.txt', 'Training_data/5/data_5_121_s1_t1_1_v7.csv_label_v1.txt','Training_data/5/data_5_121_s1_t2_1_v7.csv_label_v1.txt',
                  'Training_data/5/data_5_220_s1_t1_1_v7.csv_label_v1.txt','Training_data/5/data_5_220_s1_t2_1_v7.csv_label_v1.txt','Training_data/5/data_5_221_s1_t1_1_v7.csv_label_v1.txt','Training_data/5/data_5_221_s1_t2_1_v7.csv_label_v1.txt',
                  'Training_data/6/data_6_121_v7.csv_label_v1.txt', 'Training_data/6/data_6_221_220_v7.csv_label_v1.txt', 'Training_data/6/data_6_121_s1_t1_1_v7.csv_label_v1.txt','Training_data/6/data_6_221_s1_t1_1_v7.csv_label_v1.txt',
                  'Training_data/7/data_7_221_v7.csv_label_v1.txt','Training_data/7/data_7_221_3_v7.csv_label_v1.txt', 'Training_data/7/data_7_221_2_v7.csv_label_v1.txt', 'Training_data/7/data_7_121_s1_t1_1_v7.csv_label_v1.txt',
                  'Training_data/7/data_7_121_s2_t1_1_v7.csv_label_v1.txt','Training_data/7/data_7_121_s3_t1_1_v7.csv_label_v1.txt', 'Training_data/7/data_7_221_s1_t1_1_v7.csv_label_v1.txt',
                  'Training_data/7/data_7_221_s2_t1_1_v7.csv_label_v1.txt','Training_data/7/data_7_221_s3_t1_1_v7.csv_label_v1.txt',   
                  'Training_data/8/data_8_221_v7.csv_label_v1.txt', 'Training_data/8/data_8_221_3_v7.csv_label_v1.txt', 'Training_data/8/data_8_121_s1_t1_1_v7.csv_label_v1.txt','Training_data/8/data_8_121_s1_t2_1_v7.csv_label_v1.txt',
                  'Training_data/8/data_8_121_s1_t3_1_v7.csv_label_v1.txt', 'Training_data/8/data_8_220_s1_t1_1_v7.csv_label_v1.txt','Training_data/8/data_8_220_s1_t2_1_v7.csv_label_v1.txt','Training_data/8/data_8_220_s1_t3_1_v7.csv_label_v1.txt',
                  'Training_data/8/data_8_221_s1_t1_1_v7.csv_label_v1.txt','Training_data/8/data_8_221_s1_t2_1_v7.csv_label_v1.txt','Training_data/8/data_8_221_s1_t3_1_v7.csv_label_v1.txt',
                  'Training_data/8/data_8_221_s1_t4_1_v7.csv_label_v1.txt','Training_data/8/data_8_221_s1_t5_1_v7.csv_label_v1.txt','Training_data/8/data_8_121_s1_t4_1_v7.csv_label_v1.txt',
                  'Training_data/9/data_9_120_v7.csv_label_v1.txt', 'Training_data/9/data_9_221_220_v7.csv_label_v1.txt','Training_data/9/data_9_221_220_3_v7.csv_label_v1.txt',
                  'Training_data/9/data_9_120_s1_t1_1_v7.csv_label_v1.txt','Training_data/9/data_9_121_s1_t1_1_v7.csv_label_v1.txt','Training_data/9/data_9_220_s1_t1_1_v7.csv_label_v1.txt','Training_data/9/data_9_221_s1_t1_1_v7.csv_label_v1.txt']

data_test_filenames = [
                        'Testing_data/jundaWu_v2/data_0_jundaWu_v7.csv','Testing_data/jundaWu_v2/data_1_jundaWu_v7.csv','Testing_data/jundaWu_v2/data_2_jundaWu_v7.csv',
                        'Testing_data/jundaWu_v2/data_3_jundaWu_v7.csv','Testing_data/jundaWu_v2/data_4_jundaWu_v7.csv','Testing_data/jundaWu_v2/data_5_jundaWu_v7.csv',
                        'Testing_data/jundaWu_v2/data_6_jundaWu_v7.csv','Testing_data/jundaWu_v2/data_7_jundaWu_v7.csv','Testing_data/jundaWu_v2/data_8_jundaWu_v7.csv',
                        'Testing_data/jundaWu_v2/data_9_jundaWu_v7.csv',
                        'Testing_data/shixunWu_v2/data_0_shiXun_v7.csv','Testing_data/shixunWu_v2/data_1_shiXun_v7.csv','Testing_data/shixunWu_v2/data_2_shiXun_v7.csv',
                        'Testing_data/shixunWu_v2/data_3_shiXun_v7.csv','Testing_data/shixunWu_v2/data_4_shiXun_v7.csv','Testing_data/shixunWu_v2/data_5_shiXun_v7.csv',
                        'Testing_data/shixunWu_v2/data_6_shiXun_v7.csv','Testing_data/shixunWu_v2/data_7_shiXun_v7.csv','Testing_data/shixunWu_v2/data_8_shiXun_v7.csv',
                        'Testing_data/shixunWu_v2/data_9_shiXun_v7.csv',
                        'Testing_data/xinyunXu/data_0_XinyunXu_v7.csv','Testing_data/xinyunXu/data_1_XinyunXu_v7.csv','Testing_data/xinyunXu/data_2_XinyunXu_v7.csv',
                        'Testing_data/xinyunXu/data_3_XinyunXu_v7.csv','Testing_data/xinyunXu/data_4_XinyunXu_v7.csv','Testing_data/xinyunXu/data_5_XinyunXu_v7.csv',
                        'Testing_data/xinyunXu/data_6_XinyunXu_v7.csv','Testing_data/xinyunXu/data_7_XinyunXu_v7.csv','Testing_data/xinyunXu/data_8_XinyunXu_v7.csv',
                        'Testing_data/xinyunXu/data_9_XinyunXu_v7.csv',
                        'Testing_data/yuanhaoXie/data_0_xie_3_v7.csv','Testing_data/yuanhaoXie/data_1_xie_3_v7.csv','Testing_data/yuanhaoXie/data_2_xie_3_v7.csv',
                        'Testing_data/yuanhaoXie/data_3_xie_3_v7.csv','Testing_data/yuanhaoXie/data_4_xie_3_v7.csv','Testing_data/yuanhaoXie/data_5_xie_3_v7.csv',
                        'Testing_data/yuanhaoXie/data_6_xie_3_v7.csv','Testing_data/yuanhaoXie/data_7_xie_3_v7.csv','Testing_data/yuanhaoXie/data_8_xie_3_v7.csv',
                        'Testing_data/yuanhaoXie/data_9_xie_3_v7.csv',
                        'Testing_data/yuanhaoXie/data_0_xie_v7.csv','Testing_data/yuanhaoXie/data_1_xie_v7.csv','Testing_data/yuanhaoXie/data_2_xie_v7.csv',
                        'Testing_data/yuanhaoXie/data_3_xie_v7.csv','Testing_data/yuanhaoXie/data_4_xie_v7.csv','Testing_data/yuanhaoXie/data_5_xie_v7.csv',
                        'Testing_data/yuanhaoXie/data_6_xie_v7.csv','Testing_data/yuanhaoXie/data_7_xie_v7.csv','Testing_data/yuanhaoXie/data_8_xie_v7.csv',
                        'Testing_data/yuanhaoXie/data_9_xie_v7.csv',
                        'Testing_data/yuanhaoXie/data_0_xie_2_v7.csv', 'Testing_data/yuanhaoXie/data_3_xie_2_v7.csv',
                        'Testing_data/zehangWu/data_0_ZehangWu_v7.csv', 'Testing_data/zehangWu/data_1_ZehangWu_v7.csv', 'Testing_data/zehangWu/data_2_ZehangWu_v7.csv',
                        'Testing_data/zehangWu/data_3_ZehangWu_v7.csv', 'Testing_data/zehangWu/data_4_ZehangWu_v7.csv', 'Testing_data/zehangWu/data_5_ZehangWu_v7.csv',
                        'Testing_data/zehangWu/data_6_ZehangWu_v7.csv', 'Testing_data/zehangWu/data_7_ZehangWu_v7.csv', 'Testing_data/zehangWu/data_8_ZehangWu_v7.csv',
                        'Testing_data/zehangWu/data_9_ZehangWu_v7.csv',
                        'Testing_data/liang/data_0_Liang_v7.csv','Testing_data/liang/data_1_Liang_v7.csv','Testing_data/liang/data_2_Liang_v7.csv',
                        'Testing_data/liang/data_3_Liang_v7.csv','Testing_data/liang/data_4_Liang_v7.csv','Testing_data/liang/data_5_Liang_v7.csv',
                        'Testing_data/liang/data_6_Liang_v7.csv','Testing_data/liang/data_7_Liang_v7.csv','Testing_data/liang/data_8_Liang_v7.csv',
                        'Testing_data/liang/data_9_Liang_v7.csv',
                        'Testing_data/shixunWu/data_0_shiXun_v7.csv', 'Testing_data/shixunWu/data_1_shiXun_v7.csv','Testing_data/shixunWu/data_2_shiXun_v7.csv',
                        'Testing_data/shixunWu/data_3_shiXun_v7.csv','Testing_data/shixunWu/data_4_shiXun_v7.csv','Testing_data/shixunWu/data_5_shiXun_v7.csv',
                        'Testing_data/shixunWu/data_6_shiXun_v7.csv','Testing_data/shixunWu/data_7_shiXun_v7.csv','Testing_data/shixunWu/data_8_shiXun_v7.csv',
                        'Testing_data/shixunWu/data_9_shiXun_v7.csv'

                        ]
label_test_filenames = [
                        'Testing_data/jundaWu_v2/data_0_jundaWu_v7.csv_label_v1.txt','Testing_data/jundaWu_v2/data_1_jundaWu_v7.csv_label_v1.txt','Testing_data/jundaWu_v2/data_2_jundaWu_v7.csv_label_v1.txt',
                        'Testing_data/jundaWu_v2/data_3_jundaWu_v7.csv_label_v1.txt','Testing_data/jundaWu_v2/data_4_jundaWu_v7.csv_label_v1.txt','Testing_data/jundaWu_v2/data_5_jundaWu_v7.csv_label_v1.txt',
                        'Testing_data/jundaWu_v2/data_6_jundaWu_v7.csv_label_v1.txt','Testing_data/jundaWu_v2/data_7_jundaWu_v7.csv_label_v1.txt','Testing_data/jundaWu_v2/data_8_jundaWu_v7.csv_label_v1.txt',
                        'Testing_data/jundaWu_v2/data_9_jundaWu_v7.csv_label_v1.txt',
                        'Testing_data/shixunWu_v2/data_0_shiXun_v7.csv_label_v1.txt','Testing_data/shixunWu_v2/data_1_shiXun_v7.csv_label_v1.txt','Testing_data/shixunWu_v2/data_2_shiXun_v7.csv_label_v1.txt',
                        'Testing_data/shixunWu_v2/data_3_shiXun_v7.csv_label_v1.txt','Testing_data/shixunWu_v2/data_4_shiXun_v7.csv_label_v1.txt','Testing_data/shixunWu_v2/data_5_shiXun_v7.csv_label_v1.txt',
                        'Testing_data/shixunWu_v2/data_6_shiXun_v7.csv_label_v1.txt','Testing_data/shixunWu_v2/data_7_shiXun_v7.csv_label_v1.txt','Testing_data/shixunWu_v2/data_8_shiXun_v7.csv_label_v1.txt',
                        'Testing_data/shixunWu_v2/data_9_shiXun_v7.csv_label_v1.txt',
                        'Testing_data/xinyunXu/data_0_XinyunXu_v7.csv_label_v1.txt','Testing_data/xinyunXu/data_1_XinyunXu_v7.csv_label_v1.txt','Testing_data/xinyunXu/data_2_XinyunXu_v7.csv_label_v1.txt',
                        'Testing_data/xinyunXu/data_3_XinyunXu_v7.csv_label_v1.txt','Testing_data/xinyunXu/data_4_XinyunXu_v7.csv_label_v1.txt','Testing_data/xinyunXu/data_5_XinyunXu_v7.csv_label_v1.txt',
                        'Testing_data/xinyunXu/data_6_XinyunXu_v7.csv_label_v1.txt','Testing_data/xinyunXu/data_7_XinyunXu_v7.csv_label_v1.txt','Testing_data/xinyunXu/data_8_XinyunXu_v7.csv_label_v1.txt',
                        'Testing_data/xinyunXu/data_9_XinyunXu_v7.csv_label_v1.txt',
                        'Testing_data/yuanhaoXie/data_0_xie_3_v7.csv_label_v1.txt','Testing_data/yuanhaoXie/data_1_xie_3_v7.csv_label_v1.txt','Testing_data/yuanhaoXie/data_2_xie_3_v7.csv_label_v1.txt',
                        'Testing_data/yuanhaoXie/data_3_xie_3_v7.csv_label_v1.txt','Testing_data/yuanhaoXie/data_4_xie_3_v7.csv_label_v1.txt','Testing_data/yuanhaoXie/data_5_xie_3_v7.csv_label_v1.txt',
                        'Testing_data/yuanhaoXie/data_6_xie_3_v7.csv_label_v1.txt','Testing_data/yuanhaoXie/data_7_xie_3_v7.csv_label_v1.txt','Testing_data/yuanhaoXie/data_8_xie_3_v7.csv_label_v1.txt',
                        'Testing_data/yuanhaoXie/data_9_xie_3_v7.csv_label_v1.txt',
                        'Testing_data/yuanhaoXie/data_0_xie_v7.csv_label_v1.txt','Testing_data/yuanhaoXie/data_1_xie_v7.csv_label_v1.txt','Testing_data/yuanhaoXie/data_2_xie_v7.csv_label_v1.txt',
                        'Testing_data/yuanhaoXie/data_3_xie_v7.csv_label_v1.txt','Testing_data/yuanhaoXie/data_4_xie_v7.csv_label_v1.txt','Testing_data/yuanhaoXie/data_5_xie_v7.csv_label_v1.txt',
                        'Testing_data/yuanhaoXie/data_6_xie_v7.csv_label_v1.txt','Testing_data/yuanhaoXie/data_7_xie_v7.csv_label_v1.txt','Testing_data/yuanhaoXie/data_8_xie_v7.csv_label_v1.txt',
                        'Testing_data/yuanhaoXie/data_9_xie_v7.csv_label_v1.txt',
                        'Testing_data/yuanhaoXie/data_0_xie_2_v7.csv_label_v1.txt', 'Testing_data/yuanhaoXie/data_3_xie_2_v7.csv_label_v1.txt',
                        'Testing_data/zehangWu/data_0_ZehangWu_v7.csv_label_v1.txt', 'Testing_data/zehangWu/data_1_ZehangWu_v7.csv_label_v1.txt', 'Testing_data/zehangWu/data_2_ZehangWu_v7.csv_label_v1.txt',
                        'Testing_data/zehangWu/data_3_ZehangWu_v7.csv_label_v1.txt', 'Testing_data/zehangWu/data_4_ZehangWu_v7.csv_label_v1.txt', 'Testing_data/zehangWu/data_5_ZehangWu_v7.csv_label_v1.txt',
                        'Testing_data/zehangWu/data_6_ZehangWu_v7.csv_label_v1.txt', 'Testing_data/zehangWu/data_7_ZehangWu_v7.csv_label_v1.txt', 'Testing_data/zehangWu/data_8_ZehangWu_v7.csv_label_v1.txt',
                        'Testing_data/zehangWu/data_9_ZehangWu_v7.csv_label_v1.txt',
                        'Testing_data/liang/data_0_Liang_v7.csv_label_v1.txt','Testing_data/liang/data_1_Liang_v7.csv_label_v1.txt','Testing_data/liang/data_2_Liang_v7.csv_label_v1.txt',
                        'Testing_data/liang/data_3_Liang_v7.csv_label_v1.txt','Testing_data/liang/data_4_Liang_v7.csv_label_v1.txt','Testing_data/liang/data_5_Liang_v7.csv_label_v1.txt',
                        'Testing_data/liang/data_6_Liang_v7.csv_label_v1.txt','Testing_data/liang/data_7_Liang_v7.csv_label_v1.txt','Testing_data/liang/data_8_Liang_v7.csv_label_v1.txt',
                        'Testing_data/liang/data_9_Liang_v7.csv_label_v1.txt',
                        'Testing_data/shixunWu/data_0_shiXun_v7.csv_label_v1.txt', 'Testing_data/shixunWu/data_1_shiXun_v7.csv_label_v1.txt','Testing_data/shixunWu/data_2_shiXun_v7.csv_label_v1.txt',
                        'Testing_data/shixunWu/data_3_shiXun_v7.csv_label_v1.txt','Testing_data/shixunWu/data_4_shiXun_v7.csv_label_v1.txt','Testing_data/shixunWu/data_5_shiXun_v7.csv_label_v1.txt',
                        'Testing_data/shixunWu/data_6_shiXun_v7.csv_label_v1.txt','Testing_data/shixunWu/data_7_shiXun_v7.csv_label_v1.txt','Testing_data/shixunWu/data_8_shiXun_v7.csv_label_v1.txt',
                        'Testing_data/shixunWu/data_9_shiXun_v7.csv_label_v1.txt'

                        ]   

In [ ]:
def create_label_frame(file_name):
  label_frame = pd.DataFrame(columns = ['label'])
  for i, file in enumerate(file_name):
    data = pd.read_csv(file, names = ['label'])
    label_frame = pd.concat([label_frame, data], axis = 0, ignore_index = True)
  return label_frame

def create_data_frame(file_name):
  data_frame = []
  for i, file in enumerate(file_name):
    data = pd.read_csv(file)
    data_frame.append (data)
  data_frame = pd.concat(data_frame, join = 'outer')
  return data_frame

def data_3D_transfer(photodiode_arr, data_frame):
  data_3D = list()
  for i, photodiode_name in enumerate(photodiode_arr):
    data_3D.append(data_frame[photodiode_name].values.reshape(-1, ))  
  data_3D = dstack(data_3D)
  data_3D = data_3D.reshape((-1, 300, 9))
  return data_3D

def load_dataset():
                                         
  photodiode_arr = ['6', '1', '2', '3', '9', '8', '7', '4', '5']
  label_frame = create_label_frame(label_filenames)
  label_test_frame = create_label_frame(label_test_filenames)
  data_frame = create_data_frame(data_filenames)
  data_test_frame = create_data_frame(data_test_filenames)
  data_3D = data_3D_transfer(photodiode_arr, data_frame)
  data_test_3D = data_3D_transfer(photodiode_arr, data_test_frame)
  data_y = label_frame
  data_test_y = label_test_frame
  data_y = to_categorical(data_y,num_classes=10)
  print('data_y : ', len(data_y))
  print('data_test_y : ', len(data_test_y))
  data_test_y = to_categorical(data_test_y,num_classes=10)
  # Xtra,Xval,Ytra,Yval = train_test_split(data_3D,data_y,test_size=0.2,shuffle=True)
  return  data_3D, data_y, data_test_3D,  data_test_y
  # return  Xtra,Ytra, data_test_3D,  data_test_y, Xval,Yval

# 3. Build the model

In [3]:
learning_rate = [0.001]
verbose, epochs  = 0, 20
Batch_size = [32]

n_steps, n_length = 60, 5
num_filters = [32]
num_kernels = [3]
num_lstm_Neurons = [1024]
num_fcn_Neurons = [1024]
Attention_Neurons = [1]


n_steps_Ex, n_length_Ex = 60, 5 
num_filters_Ex = [64]
num_kernels_Ex = [2]
num_heads = [2]
num_lstm_Neurons_Ex = [512]
num_fcn_Neurons_Ex = [1024]
Attention_Neurons_Ex = [1]

In [5]:
def build_model(num_head, att_neuron, num_lstm_Neuron, num_fcn_Neuron, num_filter,num_kernel, n_timesteps, n_steps, n_length, n_features, n_outputs):
  inputs = Input(shape = (n_steps, n_length, n_features))
  TDCnn = TimeDistributed(Conv1D(filters= num_filter, kernel_size=num_kernel,  activation = 'relu'), name = 'TDCNN')(inputs)
  Pool = TimeDistributed(MaxPooling1D(pool_size =num_kernel))(TDCnn)  
  dp = TimeDistributed(Dropout(0.5))(Pool)
  flat = TimeDistributed(Flatten())(dp)
  gru = Bidirectional(GRU(num_lstm_Neuron, return_sequences = True))(flat)   
  MulselfAtt = MultiHeadAttention(head_num=num_head,name='Multi-Head')([gru, gru, gru])
  # dp1 = Dropout(0.5)(MulselfAtt)  
  fla = GlobalAveragePooling1D()(MulselfAtt)
  # gru2 = Bidirectional(GRU(num_lstm_Neuron, return_sequences=False))(MulselfAtt)
  # dp1 = Dropout(0.5)(fla)
  # den1 = Dense(num_fcn_Neuron, activation='relu')(dp1)
  # dp2 = Dropout(0.5)(den1)
  # den2 = Dense(256, activation='relu')(dp2)
  # dp3 = Dropout(0.5)(den2)  
  output = Dense(n_outputs, activation='softmax')(fla)
  model = Model(inputs = inputs, outputs = output) 
  return model  

# 4. Training the model

In [ ]:
def evaluate_model(trainX, trainy, testX, testy, valX, valy,learningRate,batch_size,num_filter, num_kernel, att_neurons, num_lstm_Neuron, num_fcn_Neuron, n_timesteps, n_steps, n_length,n_features, n_outputs):
  model = Sequential()
  model = build_model(att_neurons, num_lstm_Neuron, num_fcn_Neuron,num_filter, num_kernel,n_timesteps, n_steps, n_length, n_features, n_outputs)
  optimizer = keras.optimizers.Adam(learning_rate=learningRate) 
  model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
	# earlyStopping= EarlyStopping(monitor='val_loss', patience=50, mode='auto', verbose=1, restore_best_weights=True)
  history = model.fit(trainX, trainy, epochs=epochs, validation_data=(valX, valy), batch_size=batch_size, verbose=verbose, shuffle=True)   
	# _, accuracy = model.evaluate(testX, testy,batch_size=batch_size, verbose=0)
  model_checkpoint = ModelCheckpoint('CNNLSTMFCN_v1.hdf5', save_best_only=True, monitor='val_accuracy', mode='auto', save_weights_only=True) 
	# model_checkpoint= ModelCheckpoint('CNNLSTMFCN_v1.hdf5', save_best_only=True, monitor='val_accuracy', mode='auto', save_weights_only=True) 
  accuracy = max(history.history['val_accuracy'])
  return model, accuracy, history

def summarize_results(scores):
	print(scores)
	m, s = mean(scores), std(scores)
	print('Accuracy: %.3f%% (+/-%.3f)' % (m, s))

def run_experiment(repeats, learningRate, batch_size, num_filter, num_kernel, att_neurons, num_lstm_Neuron, num_fcn_Neuron, trainX, trainy, testX, testy, valX, valy,n_timesteps,n_steps,n_length, n_features, n_outputs, cvscores):
	# load data
	# trainX, trainy, testX, testy, valX, valy = load_dataset()
	scores = list()
	for r in range(repeats):
		# model = Sequential()
		#build_model()    
		model, score, history = evaluate_model(trainX, trainy, testX, testy, valX, valy ,learningRate, batch_size,num_filter, num_kernel, att_neurons, num_lstm_Neuron, num_fcn_Neuron,n_timesteps, n_steps, n_length,n_features, n_outputs)
		score = score * 100.0
		cvscores.append(score)
		# print('>#%d: %.3f' % (r+1, score))
		# scores.append(score)
		plt.plot(history.history['loss'])
		plt.plot(history.history['val_loss'])
		plt.title('Training Loss VS Validation Loss')
		plt.ylabel('Loss')
		plt.xlabel('epoch')
		plt.legend(['train', 'val'], loc='upper left')
		plt.show()
		print('Training Accuracy : ', max(history.history['accuracy']))   
		print('Validation Accuracy : ', max(history.history['val_accuracy']))  
		plt.plot(history.history['accuracy'])
		plt.plot(history.history['val_accuracy'])
		plt.title('Training ACC VS Validation ACC')
		plt.ylabel('Acc')
		plt.xlabel('epoch')
		plt.legend(['train', 'val'], loc='upper left')
		plt.show()  
		# if score > 90.0:
		#  break
	model.summary()  
	# summarize_results(scores)
	return model, cvscores

# 5. Initialize the SPI and TERMIO

In [6]:
def spi_init():
        spi.open(0,0)
        spi.max_speed_hz =1350000
        spi2.open(0,1)
        spi2.max_speed_hz =1350000
        
def termios_init():
        newattr[3] = newattr[3] & ~termios.ICANON & ~termios.ECHO
        termios.tcsetattr(fd, termios.TCSANOW, newattr)
        fcntl.fcntl(fd, fcntl.F_SETFL, oldflags | os.O_NONBLOCK)

# 6. Timer Class

In [7]:
class RepeatedTimer(object):
  def __init__(self, interval, function, *args, **kwargs):
    self._timer = None
    self.interval = interval
    self.function = function
    self.args = args
    self.kwargs = kwargs
    self.is_running = False
    self.next_call = time.time()
    self.start()

  def _run(self):
    self.is_running = False
    self.start()
    self.function(*self.args, **self.kwargs)

  def start(self):
    if not self.is_running:
      self.next_call += self.interval
      self._timer = threading.Timer(self.next_call - time.time(), self._run)
      self._timer.start()
      self.is_running = True

  def stop(self):
    self._timer.cancel()
    self.is_running = False

# 7. Read Sensor Data

In [8]:
def get_char():
    fd = sys.stdin.fileno()
    old_settings = termios.tcgetattr(fd)
    try:
        tty.setraw(sys.stdin.fileno())
        ch = sys.stdin.read(1)
    finally:
        termios.tcsetattr(fd, termios.TCSADRAIN, old_settings)
    return ch


def analogInput_CE0(channel,spi):
    adc = spi.xfer2([1, (8+channel)<<4, 0])
    data = ((adc[1]&3) <<8) + adc[2]
    return data

def analogInput_CE1(channel,spi2):
    adc = spi2.xfer2([1, (8+channel)<<4, 0])
    data = ((adc[1]&3) <<8) + adc[2]
    return data

def read_data():
        if len(output)  < 500:
            aux = []
#             aux.append(time.time())
            aux.append(analogInput_CE1(2,spi2))
            aux.append(analogInput_CE0(3,spi))
            aux.append(analogInput_CE0(7,spi))
            aux.append(analogInput_CE1(3,spi2))
            aux.append(analogInput_CE1(1,spi2))
            aux.append(analogInput_CE0(5,spi))
            aux.append(analogInput_CE0(1,spi))
            aux.append(analogInput_CE0(2,spi))
            aux.append(analogInput_CE0(6,spi))
            output.append(aux)
        

# load and repshAPE DATA

In [ ]:
trainX, trainy, valX, valy = load_dataset()
n_timesteps, n_features, n_outputs = trainX.shape[1], trainX.shape[2], trainy.shape[1]

In [ ]:
trainX1 = trainX.reshape((trainX.shape[0], n_steps, n_length, n_features)) 
valX1 = valX.reshape((valX.shape[0],n_steps, n_length, n_features)) 

In [ ]:
cvscores = []
for att_neurons in Attention_Neurons:
  for j in range(len(num_fcn_Neurons)):
    for num_kernel in num_kernels:
      for num_filter in num_filters:  
        for learningRate in learning_rate:     
          for batch_size in Batch_size:
            for i in range(len(num_lstm_Neurons)): 
              print('n_steps : ', n_steps)
              print('n_length : ', n_length)
              print('batch_size :  ', batch_size)
              print('learningRate :  ', learningRate)
              print('num_filter :  ', num_filter)
              print('num_kernel :  ', num_kernel)
              print('num_fcn_Neurons :  ', num_fcn_Neurons[j])
              print('num_lstm_neurons :  ', num_lstm_Neurons[i]) 
              print('num_Att_neurons : ', att_neurons)          
              prev_time = time.time()
              model, cvscores = run_experiment(repeats=1, learningRate= learningRate, batch_size = batch_size, num_filter= num_filter, num_kernel = num_kernel, att_neurons = att_neurons, num_lstm_Neuron = num_lstm_Neurons[i], num_fcn_Neuron = num_fcn_Neurons[j],trainX= trainX1, trainy=trainy, testX=None, testy= None, valX=valX1, valy= valy, n_timesteps = n_timesteps, n_steps=n_steps, n_length = n_length, n_features=n_features, n_outputs=n_outputs, cvscores = cvscores)
              print('Interval is : ', time.time() - prev_time)
              print('-----------------------------------------')

# SAVE MODEL

In [ ]:
from tensorflow.keras.models import save_model, load_model
filepath = './CNNLSTMFCN_v3'
save_model(model, filepath, save_format='h5')
# model = load_model('saved_model4')

In [8]:
from keras.models import save_model, load_model
# filepath = './saved_model'
# save_model(model, filepath, save_format='h5')
model = load_model("CNNLSTMFCN_v1")

In [10]:
learning_rate = [0.001]
verbose, epochs  = 0, 20
Batch_size = [32]

n_steps, n_length = 60, 5
num_filters = [32]
num_kernels = [3]
num_lstm_Neurons = [ 1024]
num_heads = [2]
num_fcn_Neurons = [1024]
Attention_Neurons = [1]

model = build_model(num_heads[0], Attention_Neurons[0], num_lstm_Neurons[0], num_fcn_Neurons[0],num_filters[0], num_kernels[0],300, n_steps, n_length, 9, 10)
model.load_weights("All_asTrain_and_Test_TF20_ACC92.hdf5")

In [ ]:
def isCovered(data, digit):
    if data[data.digit < 150].count() > 170:
        return True
    else: 
        return False

def calibration_mode():
    print("Calibration Mode: Please put your finger tip on the center photodiode  ")
    data_cal = pd.DataFrame(output[300:500], columns = ['6', '1', '2', '3', '9', '8', '7', '4', '5'])
    max_val = data_cal.max()
    print('max_val : ', max_val)
    if isCovered(data_cal, 2):
        print ("sensing area is : 3; data_reading area: 1" )
    elif isCovered(data_cal, 1):
        print ("sensing area is : 4; data_reading area: 1" )     
    elif isCovered(data_cal, 3):
        print ("sensing area is : 3; data_reading area: 2" )
    elif isCovered(data_cal, 4) and (not isCovered(data_cal, 5) ):
        print ("sensing area is : 2; data_reading area: 1" )   
    elif isCovered(data_cal, 6) and (not isCovered(data_cal, 5) ):
        print ("sensing area is : 1; data_reading area: 2" )
    elif isCovered(data_cal, 8) and (not isCovered(data_cal, 5) ):
        print ("sensing area is : 1; data_reading area: 3" )
    elif isCovered(data_cal, 7) and (not isCovered(data_cal, 5) ):
        print ("sensing area is : 2; data_reading area: 3" )
    elif isCovered(data_cal, 9) and (not isCovered(data_cal, 5) ):
        print ("sensing area is : 1; data_reading area: 4" )
    else:
        print ("sensing area is : 1; data_reading area: 1" )   
    return True

In [ ]:
from scipy.signal import savgol_filter
from sklearn import preprocessing
import cv2

def min_max_scale(X, range=(0, 100)):
    mi, ma = range
    max_list = X.max()
    min_list = X.min()
    max_val = max_list.max(axis=0)
    min_val = min_list.min(axis=0)
    X_std = (X - min_val) / (max_val - min_val)
    X_scaled = X_std * (ma - mi) + mi
    return X_scaled

if __name__ == '__main__':  
    global spi, spi2, output,fd, oldterm, newattr, oldflags, real_data_3D
    spi = spidev.SpiDev()
    spi2 = spidev.SpiDev()
    output = []
    spi_init()
    RepeatedTimer(0.01,read_data).start()
    
    while 1: 
        if calibration_mode():
            break
        else:
            print("Error: ...")
        
    while 1:
        inkey = input()
        if inkey == 'n':
            print('start get input')
            output = []
            sleep(5.5)
#             output_3D = []
#             output_3D.append(output[0:500])
#             real_data_3D = np.array(output_3D)
#             print(real_data_3D.shape)
            # 1. remove outliers
#             print(output.shape)
            data_v1 = pd.DataFrame(output[0:500], columns = ['6', '1', '2', '3', '9', '8', '7', '4', '5'])
            print(data_v1.head(1))
            Q1 = data_v1.quantile(0.01)
            Q3 = data_v1.quantile(0.99)
            for m in range(1, 10, 1):
                data_v1.loc[data_v1[str(m)] > Q3[str(m)], str(m)] = Q3[str(m)]
                data_v1.loc[data_v1[str(m)] < Q1[str(m)], str(m)] = Q1[str(m)]
            data_v2 = savgol_filter(data_v1[['6', '1', '2', '3', '9', '8', '7', '4', '5']], 35, 7, axis = 0)
            data_v2 = pd.DataFrame(data_v2, columns = ['6', '1', '2', '3', '9', '8', '7', '4', '5'])
#             print('------------------')
#             print(data_v2.head(10))            
            max_list = data_v2.max()
            max_val = max_list.max(axis = 0)
            diff_list = max_val - max_list
            for m in range(1, 10, 1):
                data_v2.loc[:, str(m)] = data_v2.loc[:, str(m)] + diff_list[str(m)]
            data_v3 = min_max_scale(data_v2)
#             print('------------------')
#             print(data_v3.head(10))
            data_v3 = pd.DataFrame(data_v3, columns = ['6', '1', '2', '3', '9', '8', '7', '4', '5'])            
            start_point = data_v3[((data_v3['1'] <= 70) & (data_v3['1'] > 0)) | 
                                  ((data_v3['2'] <= 70) & (data_v3['2'] > 0)) | 
                                  ((data_v3['3'] <= 70) & (data_v3['3'] > 0)) | 
                                  ((data_v3['4'] <= 70) & (data_v3['4'] > 0)) | 
                                  ((data_v3['5'] <= 70) & (data_v3['5'] > 0)) | 
                                  ((data_v3['6'] <= 70) & (data_v3['6'] > 0)) | 
                                  ((data_v3['7'] <= 70) & (data_v3['7'] > 0)) | 
                                  ((data_v3['8'] <= 70) & (data_v3['8'] > 0)) | 
                                  ((data_v3['9'] <= 70) & (data_v3['9'] > 0)) ]
            if (start_point.empty):
                break
            start = int(start_point.index[0])
            end = int(start_point.index[-1])
            if (start - 10 >= 0):
                start -= 10
            if (end + 10 <= 499):
                end += 10
            pic = cv2.resize(data_v3.loc[start: end, ['6', '1', '2', '3', '9', '8', '7', '4', '5']].values, (9, 300), interpolation =  cv2.INTER_LINEAR)
            start_point.iloc[0:0]
            data_v4 = pd.DataFrame(pic, columns = ['6', '1', '2', '3', '9', '8', '7', '4', '5'])
#             print(data_v4.shape)
#             print('------------------')
#             print(data_v4.head(10))
            data_v4.values.reshape((-1, 300, 9))
            real_data_3D = data_v4.values.reshape((-1, 60, 5, 9))
#             print(real_data_3D)
            label = model.predict(real_data_3D)
#             print("label is ", label)
            print('label is ', np.argmax(label))

n
start get input
     6    1    2    3    9    8    7    4    5
0  259  239  228  358  263  263  269  262  262
label is  0
n
start get input
     6    1    2    3    9    8    7    4    5
0  246  248  244  344  258  260  260  232  251


In [ ]:
a = input()